In [8]:
#%% Packages
import torch
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import pandas as pd 
import random
import os

In [9]:
use_gpu = False  # set it to True to use GPU and False to use CPU
if use_gpu:
    torch.cuda.set_device(0)

In [10]:
def seed_everything(seed=40):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

## Data preparation

In [11]:
df = pd.read_csv('ADNI.csv')#reading data
df = df.fillna(0)
data=df.values 
data_label = (data[:,1].reshape(-1,1))    
Data = data[:,2:-1]
data_label = (data_label-data_label.min())/(data_label.max()-data_label.min()) 

In [12]:
df = pd.read_csv('ADNI.csv') # Computing the missing matrix
data=df.values
Missing = df.fillna(-1).values[:,2:-1]
Missing[Missing>-0.5]=1
Missing[Missing<-0.5]=0

In [13]:
labels = np.zeros([1167])    #Get data for which missing pattern
Missing=np.array(Missing)
Missing_labels = {}
Missing_tabel = {}
k=0
for i in range(Missing.shape[0]):
    if tuple(Missing[i][:])not in Missing_labels:
        Missing_labels[tuple(Missing[i][:])] = k
        k = k+1 
    if tuple(Missing[i][:])not in Missing_tabel:
        Missing_tabel[tuple(Missing[i][:])] = 1
    if tuple(Missing[i][:]) in Missing_tabel:
        Missing_tabel[tuple(Missing[i][:])] +=1
        
    labels[i] = Missing_labels[tuple(Missing[i][:])]
labels = labels.reshape(1167,1)

## Hyper-parameters

In [14]:
#%% System Parameters
# 1. Mini batch size
mb_size = 64
# 2.hint rate
p_hint=0.8
#3.random seed 
seed_everything(49)
# 4.K pattern
k_pattern=8
# 5. Loss Hyperparameters
alpha = 10
# 6. Train Rate
train_rate = 0.8
#7.specifiy gamma here or use CV to select gamma
gamma=0.5
# Parameters
No = len(Data)
Dim = len(Data[0,:])

# Hidden state dimensions
H_Dim1 = Dim
H_Dim2 = Dim
label_pattern = 1 
#For multi-classification problems, there are multiple label patterns

# Normalization (0 to 1)
Min_Val = np.zeros(Dim)
Max_Val = np.zeros(Dim)

for i in range(Dim):
    Min_Val[i] = np.min(Data[:,i])
    Data[:,i] = Data[:,i] - np.min(Data[:,i])
    Max_Val[i] = np.max(Data[:,i])
    Data[:,i] = Data[:,i] / (np.max(Data[:,i]) + 1e-6)    

In [15]:
#%% Train Test Division    
   
idx = np.random.permutation(No)

Train_No = int(No * train_rate)
Test_No = No - Train_No
    
# Train / Test Features
trainX = Data[idx[:Train_No],:]
testX = Data[idx[Train_No:],:]

# Train / Test Missing Indicators
trainM = Missing[idx[:Train_No],:]
testM = Missing[idx[Train_No:],:]

#Train/Test Pattern Label
trainlabel = labels[idx[:Train_No]]
testlabel = labels[idx[Train_No:]]

#Train/Test Data Label
traindatalabel = data_label[idx[:Train_No]]
testdatalabel = data_label[idx[Train_No:]]

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size = size, scale = xavier_stddev)
    
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C

##  Network Architecture

In [16]:
#%% 1. Discriminator
if use_gpu is True:
    D_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True, device="cuda")     # Data+ Hint as inputs
    D_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True, device="cuda")

    D_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True, device="cuda")
    D_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True, device="cuda")

    D_W3 = torch.tensor(xavier_init([H_Dim2, k_pattern]),requires_grad=True, device="cuda")
    D_b3 = torch.tensor(np.zeros(shape = [k_pattern]),requires_grad=True, device="cuda")       # Output is multi-variate
else:
    D_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True)     # Data + Hint as inputs
    D_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True)

    D_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True)
    D_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True)

    D_W3 = torch.tensor(xavier_init([H_Dim2, k_pattern]),requires_grad=True)
    D_b3 = torch.tensor(np.zeros(shape = [k_pattern]),requires_grad=True)       # Output is multi-variate

theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]

#%% 2. Generator
if use_gpu is True:
    G_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True, device="cuda")     # Data + Mask as inputs (Random Noises are in Missing Components)
    G_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True, device="cuda")

    G_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True, device="cuda")
    G_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True, device="cuda")

    G_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True, device="cuda")
    G_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True, device="cuda")
else:
    G_W1 = torch.tensor(xavier_init([Dim*2, H_Dim1]),requires_grad=True)     # Data + Mask as inputs (Random Noises are in Missing Components)
    G_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True)

    G_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2]),requires_grad=True)
    G_b2 = torch.tensor(np.zeros(shape = [H_Dim2]),requires_grad=True)

    G_W3 = torch.tensor(xavier_init([H_Dim2, Dim]),requires_grad=True)
    G_b3 = torch.tensor(np.zeros(shape = [Dim]),requires_grad=True)


# 3.Predictor
if use_gpu is True:
    P_W1 = torch.tensor(xavier_init([Dim, H_Dim1]),requires_grad=True, device="cuda")     # Data as inputs
    P_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True, device="cuda")

    P_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2//2]),requires_grad=True, device="cuda")
    P_b2 = torch.tensor(np.zeros(shape = [H_Dim2//2]),requires_grad=True, device="cuda")

    P_W3 = torch.tensor(xavier_init([H_Dim2//2, label_pattern]),requires_grad=True, device="cuda")
    P_b3 = torch.tensor(np.zeros(shape = [label_pattern]),requires_grad=True, device="cuda")       # Output is multi-variate
else:
    P_W1 = torch.tensor(xavier_init([Dim, H_Dim1]),requires_grad=True)     # Data  as inputs
    P_b1 = torch.tensor(np.zeros(shape = [H_Dim1]),requires_grad=True)

    P_W2 = torch.tensor(xavier_init([H_Dim1, H_Dim2//2]),requires_grad=True)
    P_b2 = torch.tensor(np.zeros(shape = [H_Dim2//2]),requires_grad=True)

    P_W3 = torch.tensor(xavier_init([H_Dim2//2, label_pattern]),requires_grad=True)
    P_b3 = torch.tensor(np.zeros(shape = [label_pattern]),requires_grad=True)       # Output is multi-variate
    
theta_G = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
theta_D = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
theta_P = [P_W1, P_W2, P_W3, P_b1, P_b2, P_b3, G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]  #Joint training G and P
# theta_P = [P_W1, P_W2, P_W3, P_b1, P_b2, P_b3]                                    # Only training P

## FragmGAN Functions

In [17]:
#%% 1. Generator
def generator(new_x,m):
    inputs = torch.cat(dim = 1, tensors = [new_x,m])  # Mask + Data Concatenate
    G_h1 = F.relu(torch.matmul(inputs, G_W1) + G_b1)
    G_h2 = F.relu(torch.matmul(G_h1, G_W2) + G_b2)   
    G_prob = torch.sigmoid(torch.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob

#%% 2. Discriminator
def discriminator(new_x, h):
    inputs = torch.cat(dim = 1, tensors = [new_x,h])                   # Hint + Data Concatenate                                               
    D_h1 = F.relu(torch.matmul(inputs, D_W1) + D_b1)  
    D_h2 = F.relu(torch.matmul(D_h1, D_W2) + D_b2)
    D_logit = torch.matmul(D_h2, D_W3) + D_b3
    D_prob = torch.softmax(D_logit,dim=1)  # [0,1] Probability Output
    return D_prob

# 3 Predictor 
def predictor(imput_x):
    inputs = imput_x
    P_h1 = F.relu(torch.matmul(inputs, P_W1) + P_b1)  
    P_h2 = F.relu(torch.matmul(P_h1, P_W2) + P_b2)
    P_logit = torch.matmul(P_h2, P_W3) + P_b3
    P_prob = torch.sigmoid(P_logit)  # [0,1] Probability Output
    return P_prob


def sample_Z(m, n):
    return np.random.uniform(0., 0.01, size = [m, n])        

# Mini-batch generation
def sample_idx(m, n):
    A = np.random.permutation(m)
    idx = A[:n]
    return idx

## FragmGAN Losses

In [18]:
def discriminator_loss(M, New_X, H,label):
    # Generator
    G_sample = generator(New_X,M)
    # Combine with original data
    Hat_New_X = New_X * M + G_sample * (1-M)
    # Discriminator
    D_prob = discriminator(Hat_New_X, H)
    #%% Loss
    D_loss = -torch.mean(label * torch.log(D_prob + 1e-8) + (1-label) * torch.log(1.-D_prob + 1e-8))

    return D_loss

def generator_loss(X, M, New_X, H,label):
    G_sample = generator(New_X,M)
    Hat_New_X = New_X * M + G_sample * (1-M)
    D_prob = discriminator(Hat_New_X, H)
    G_loss1 = torch.mean(label * torch.log(D_prob + 1e-8) + (1-label) * torch.log(1.-D_prob + 1e-8))
    MSE_train_loss = torch.mean((M * New_X - M * G_sample)**2) / torch.mean(M)

    G_loss = G_loss1 + alpha * MSE_train_loss 

    #%% MSE Performance metric
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return G_loss, MSE_train_loss, MSE_test_loss

def predictor_loss(imput_data,data_label):
    imput_prob = predictor(imput_data)
    P_loss = torch.mean((imput_prob-data_label)**2)
    return P_loss, imput_prob

def test_loss(X, M, New_X):
    G_sample = generator(New_X,M)
    Imput_data = M*X + (1-M)*G_sample
    MSE_test_loss = torch.mean(((1-M) * X - (1-M)*G_sample)**2) / torch.mean(1-M)
    return MSE_test_loss, Imput_data

def Mse(y_pred,y_label):
    loss = torch.mean((y_pred-y_label)**2)
    return loss


## Optimizers

In [19]:
optimizer_D = torch.optim.Adam(params=theta_D)
optimizer_G = torch.optim.Adam(params=theta_G)
optimizer_P = torch.optim.Adam(params=theta_P)

## Training

In [20]:
#%% Start Iterations
best_test_loss=1
best_test_mse=1
best_test_loss_cur=1
for it in tqdm(range(5000)):    
    #%% Inputs
    mb_idx = sample_idx(Train_No, mb_size)
    X_mb = trainX[mb_idx,:]
    label = trainlabel[mb_idx]
    data_label = traindatalabel[mb_idx]
    
    Z_mb = sample_Z(mb_size, Dim) 
    M_mb = trainM[mb_idx,:]  
    H_mb1 = sample_M(mb_size, Dim, 1-p_hint)
    H_mb = M_mb * H_mb1
    
    New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
    
    if use_gpu is True:
        X_mb = torch.tensor(X_mb, device="cuda")
        M_mb = torch.tensor(M_mb, device="cuda")
        H_mb = torch.tensor(H_mb, device="cuda")
        New_X_mb = torch.tensor(New_X_mb, device="cuda")
        label = torch.tensor(label,device='cuda').long()
        data_label = torch.tensor(data_label, device="cuda")
    else:
        X_mb = torch.tensor(X_mb)
        M_mb = torch.tensor(M_mb)
        H_mb = torch.tensor(H_mb)
        New_X_mb = torch.tensor(New_X_mb)
        label = torch.tensor(label)
        data_label = torch.tensor(data_label)
    
    #optimizer D 
    
    optimizer_D.zero_grad()
    D_loss_curr = discriminator_loss(M=M_mb, New_X=New_X_mb, H=H_mb,label=label)
    D_loss_curr.backward()
    optimizer_D.step()
    
    #optimizer G
    optimizer_G.zero_grad()
    G_loss_curr, MSE_train_loss_curr, MSE_test_loss_curr = generator_loss(X=X_mb, M=M_mb, New_X=New_X_mb, H=H_mb,label=label)
    G_loss_curr.backward()
    optimizer_G.step()    
    Z_mb_ = sample_Z(Train_No, Dim) 
    M_mb_ = trainM
    X_mb_ = trainX

    New_X_mb_ = M_mb_ * X_mb_ + (1-M_mb_) * Z_mb_  # Missing Data Introduce

    if use_gpu is True:
        X_mb_ = torch.tensor(X_mb_, device='cuda')
        M_mb_ = torch.tensor(M_mb_, device='cuda')
        New_X_mb_ = torch.tensor(New_X_mb_, device='cuda')
    else:
        X_mb_ = torch.tensor(X_mb_)
        M_mb_ = torch.tensor(M_mb_)
        New_X_mb_ = torch.tensor(New_X_mb_)

    MSE_final, Sample = test_loss(X=X_mb_, M=M_mb_, New_X=New_X_mb_)
    imput_data = generator(New_X_mb_,M_mb_)
    
    #optimizer P
    optimizer_P.zero_grad()
    P_loss_curr,imput_prob= predictor_loss(imput_data=imput_data, data_label=torch.tensor(traindatalabel))
    P_loss_curr = 1*P_loss_curr
    P_loss_curr.backward()
    optimizer_P.step()
    
    
    #Calculate MSE from test data
    
    test_Z_mb_ = sample_Z(Test_No, Dim) 
    test_M_mb_ = testM
    test_X_mb_ = testX
    test_label = testdatalabel
    
    test_New_X_mb_ = test_M_mb_ * test_X_mb_ + (1-test_M_mb_) * test_Z_mb_  # Missing Data Introduce

    if use_gpu is True:
        test_X_mb_ = torch.tensor(test_X_mb_, device='cuda')
        test_M_mb_ = torch.tensor(test_M_mb_, device='cuda')
        test_New_X_mb_ = torch.tensor(test_New_X_mb_, device='cuda')
        test_label = torch.tensor(test_label, device='cuda')
    else:
        test_X_mb_ = torch.tensor(test_X_mb_)
        test_M_mb_ = torch.tensor(test_M_mb_)
        test_New_X_mb_ = torch.tensor(test_New_X_mb_)
        test_label = torch.tensor(test_label)
        
    test_data = generator(test_New_X_mb_,test_M_mb_)
    
    test_loss_curr,test_prob= predictor_loss(imput_data=test_data, data_label=test_label)
    
    test_mse = Mse(test_label, test_prob)
    
    if best_test_mse>test_mse:
        best_test_mse = test_mse
    
    #%% Intermediate Losses
    if (it+1) % 5000 == 0:
        print('Iter: {}'.format(it))
        print('Test_best_mse:',best_test_mse)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:53<00:00, 93.61it/s]

Iter: 4999
Test_best_mse: tensor(0.0073, dtype=torch.float64, grad_fn=<MeanBackward0>)
